**Questions**

- Combien d’ID RNB sont partagés entre le bailleur et PrioRéno ?
- Combien sont juste dans un des deux jeu de données ?
- Est-ce que pour un ID RNB donné le bailleur social et PrioRéno LS parlent du même bâtiment physique.

In [1]:
import pandas as pd

bailleur_path = 'LMH_par_LMH.xlsx'
prioreno_path = 'LMH_dans_PrioReno.xlsx'

## Colonnes dispo

In [2]:
df_bailleur_raw = pd.read_excel(bailleur_path)
df_prioreno_raw = pd.read_excel(prioreno_path)

print("Colonnes - bailleur :")
print(df_bailleur_raw.columns.tolist())
print()

print("Colonnes - PrioRéno :")
print(df_prioreno_raw.columns.tolist())

Colonnes - bailleur :
['Libellé', 'HP1', 'HP2', 'HP3', 'VILLE', 'ADRESSE IKOS RECTIFIE', 'ADRESSE BAN', 'BAN.properties.id', 'RNB']

Colonnes - PrioRéno :
['Siren', 'Région', 'Code département', 'Département', 'Adresse principale', 'Commune', 'Code postal', "Nombre d'étages", 'Nombre de logements sociaux', 'Nombre total de logements', 'Identifiant RNB', 'Latitude', 'Longitude']


In [3]:
# === 1. Normalisation côté bailleur ===
df_bailleur = df_bailleur_raw.rename(columns={"RNB": "ID_RNB"}).copy()

# ID en chaînes propres, sans espaces, et suppression des vides
df_bailleur["ID_RNB"] = (
    df_bailleur["ID_RNB"]
    .astype(str)
    .str.strip()
    .replace({"": pd.NA, "nan": pd.NA})
)
df_bailleur = df_bailleur.dropna(subset=["ID_RNB"])

print("Colonnes bailleur :", df_bailleur.columns.tolist())
print("Nb de lignes bailleur après nettoyage :", len(df_bailleur))
print("Le fichier LMH a pour unité les entrées (col HP3) de bâtiments (col HP2) dans une résidence (col HP1)")

# === 2. Normalisation côté PrioRéno ===
df_prioreno = df_prioreno_raw.rename(columns={"Identifiant RNB": "ID_RNB"}).copy()

# Remplacer les NaN par chaîne vide, trim, puis split sur les espaces
df_prioreno["ID_RNB"] = (
    df_prioreno["ID_RNB"]
    .fillna("")          # NaN -> ""
    .astype(str)
    .str.strip()
)

# On transforme chaque cellule en liste d'IDs (split sur les espaces, y compris multiples)
df_prioreno["ID_RNB"] = df_prioreno["ID_RNB"].str.split()

# On "explose" pour avoir une ligne par ID_RNB
df_prioreno = df_prioreno.explode("ID_RNB")

# Nettoyage : remove vides éventuels
df_prioreno["ID_RNB"] = (
    df_prioreno["ID_RNB"]
    .astype(str)
    .str.strip()
    .replace({"": pd.NA, "nan": pd.NA})
)
df_prioreno = df_prioreno.dropna(subset=["ID_RNB"])

print(" ")
print("Colonnes PrioRéno :", df_prioreno.columns.tolist())
print("Nb de lignes PrioRéno après éclatement/clean :", len(df_prioreno))

Colonnes bailleur : ['Libellé', 'HP1', 'HP2', 'HP3', 'VILLE', 'ADRESSE IKOS RECTIFIE', 'ADRESSE BAN', 'BAN.properties.id', 'ID_RNB']
Nb de lignes bailleur après nettoyage : 5355
Le fichier LMH a pour unité les entrées (col HP3) de bâtiments (col HP2) dans une résidence (col HP1)
 
Colonnes PrioRéno : ['Siren', 'Région', 'Code département', 'Département', 'Adresse principale', 'Commune', 'Code postal', "Nombre d'étages", 'Nombre de logements sociaux', 'Nombre total de logements', 'ID_RNB', 'Latitude', 'Longitude']
Nb de lignes PrioRéno après éclatement/clean : 2391


In [5]:
# Combien de lignes LMH contiennent un ID RNB également présent dans PrioRéno ?

# Ensemble des IDs PrioRéno
ids_prioreno = set(df_prioreno["ID_RNB"])

# Sélection des lignes bailleur dont l'ID existe dans PrioRéno
df_bailleur_matched = df_bailleur[df_bailleur["ID_RNB"].isin(ids_prioreno)]



print(f"Nombre de lignes LMH avec un ID RNB présent dans PrioRéno :", len(df_bailleur_matched))
print("Nombre de lignes dans LMH : ", len(df_bailleur))
print("Pourcentage : ", (len(df_bailleur_matched) / len(df_bailleur) * 100 ))

Nombre de lignes LMH avec un ID RNB présent dans PrioRéno : 3246
Nombre de lignes dans LMH :  5355
Pourcentage :  60.61624649859944


In [16]:
# Extrait des lignes qui matchent

df_joint = df_bailleur_matched.merge(
    df_prioreno,
    on="ID_RNB",
    how="left",
    suffixes=("_LMH", "_PR")
)

# Adresse LMH : ADRESSE IKOS RECTIFIE + VILLE
df_joint["ADRESSE_LMH"] = (
    df_joint["ADRESSE IKOS RECTIFIE"].astype(str).str.strip()
)

# Adresse PrioRéno : Adresse principale + CP + Commune
df_joint["ADRESSE_PR"] = (
    df_joint["Adresse principale"].astype(str).str.strip()
)

# =========================
# 4. Extrait des lignes qui matchent
# =========================
extrait = df_joint[[
    "ID_RNB",
    "ADRESSE_LMH",
    "ADRESSE_PR",
]].sample(n=100, random_state=42)

print(extrait.to_string(index=False))

      ID_RNB                                       ADRESSE_LMH                                            ADRESSE_PR
4PC5G7KVDB2X                   44  RUE  DU FOUR A CHAUX, LILLE                    44 RUE DU FOUR A CHAUX 59000 LILLE
XTBD73CZFZRW        102  RUE  DES COULONS PROLONGEE, TOURCOING              102 Rue des Coulons prolongée  TOURCOING
RGP71DYNHK6B                28  RUE DE L AMIRAL COURBET, LILLE                24 RUE DE L AMIRAL COURBET 59800 LILLE
7FH14RCAD82F                            12  RUE CABANIS, LILLE                              0002B RUE CABANIS  LILLE
RWPHHHTCJ129         57  RUE DES CHERCHEURS, VILLENEUVE D ASCQ          1 RUE DES CHERCHEURS 59650 VILLENEUVE D ASCQ
61JPXCKEY8E6                     30  RUE  ROGER SALENGRO, LOOS                      30 RUE ROGER SALENGRO 59120 LOOS
JN15NA2MA4WX                      152  RUE DU COLLEGE, ROUBAIX                      152 RUE DU COLLEGE 59100 ROUBAIX
FQT8C8D5MY39                        88  RUE FRAGONARD, ROUBAIX  

In [6]:
# Extrait des lignes LMH qui ne matchent pas

df_bailleur_unmatched = df_bailleur[~df_bailleur["ID_RNB"].isin(ids_prioreno)]

# Random sample of 50 (or fewer if not available)
sample_unmatched = df_bailleur_unmatched.sample(
    n=min(50, len(df_bailleur_unmatched)),
    random_state=42
)

# Optional: keep only useful fields
sample_unmatched = sample_unmatched[[
    "ID_RNB",
    "HP1", "HP2", "HP3",
    "ADRESSE IKOS RECTIFIE",
    "VILLE"
]]

print(sample_unmatched.to_string(index=False))

      ID_RNB  HP1  HP2  HP3                                ADRESSE IKOS RECTIFIE                     VILLE
3X6BNPKMPCBS T010 0003 0101                   53  RUE PAUL VERSCHAEVE, TOURCOING                 TOURCOING
S3W9G3GA8X23 1140   01   15                                 15  RUE MONGE, LILLE                     LILLE
BE55N9S864H5 1450   01   21                        21  RUE PIERRE LEGRAND, LILLE                     LILLE
G1FMKMZHAH13 T015 0001 0089                   186  RUE ROBERT SCHUMAN, TOURCOING                 TOURCOING
DZKA2235N2Y8 T009 0001 0003                   6  RUE DU DOCTEUR NAERT, TOURCOING                 TOURCOING
ZNSFNM7NJE54 R026 0003 0001              1  PLACE JEAN BAPTISTE CLEMENT, ROUBAIX                   ROUBAIX
9CV78BWM42ZY T015 0001 0049                     98  RUE MARCEL BEYENS, TOURCOING                 TOURCOING
YKFK657YX3RT R939 0003 0150                       150  RUE  DU FONTENOY, ROUBAIX                   ROUBAIX
DTFJW6WVKWZS 3720 0001 0002          

In [15]:
# Chercher des lignes LMH dans une certaine rue

import pandas as pd

# Liste des sous-chaînes à chercher
required = ["ROLLAND", "20"]  # à adapter

# Colonne d'adresse convertie en string
addr = df_bailleur["ADRESSE IKOS RECTIFIE"].astype(str)

# Masque initial : tout à True
mask = pd.Series(True, index=df_bailleur.index)

# On impose la présence de chaque sous-chaîne
for s in required:
    mask &= addr.str.contains(s, case=False, na=False)

# Filtrage
df_filtered = df_bailleur[mask]

print(df_filtered.to_string(index=False))


Libellé  HP1  HP2  HP3   VILLE           ADRESSE IKOS RECTIFIE                         ADRESSE BAN BAN.properties.id       ID_RNB
ANSEELE R008 0002 0020 ROUBAIX 20  RUE ROMAIN ROLLAND, ROUBAIX 20 Rue Romain Rolland 59100 Roubaix  59512_4836_00020 YNPDNWWV4Z6G
